In [18]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import os

In [2]:
spark = SparkSession.builder.appName("extract_emvar").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/10 14:12:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
chromosome="NONE"
#chromosome="chr22"

if "which_chr" in os.environ:
    chromosome=os.environ['which_chr']

if chromosome=="NONE":
    print("error : did not find which chromosome we are supposed to crunch")
    exit(-1)
else:
    print("only crunching chromosome "+chromosome)

only crunching chromosome chr22


24/05/10 14:12:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [5]:
variant_path=f"/home/mcn26/varef/scripts/noon_data/2.0.annotate/annotated_output_{chromosome}.csv.gz/*.csv.gz"
variants=spark.read.option("delimiter","\t").csv(variant_path, header=True, inferSchema=True)

In [6]:
#remove bad variants
#Note that this filtering is redundant with `3.0.pleio_and_filter`, but it is being performed on an earlier step : so filtering such as is performed in `2.5.filter` is not done. 
variants=variants.filter(F.col("category")!="MAF_OR_AC_IS_ZERO")

In [10]:
#remove variants without malin predictions (indels)
variants=variants.dropna(how="any",subset=["K562__ref","HepG2__ref","SKNSH__ref",
                                  "K562__alt","HepG2__alt","SKNSH__alt",
                                  "K562__skew","HepG2__skew","SKNSH__skew"])

In [11]:
#first we compute whether each variant can be called an emvar in each cell-type. 
cell_types=["K562","SKNSH","HepG2"]
for cell_type in cell_types:
    variants = variants.withColumn(f"emVar_{cell_type}", 
                           (F.abs(F.col(f"{cell_type}__skew")) >= 0.5) & 
                           (F.greatest(F.col(f"{cell_type}__ref"), F.col(f"{cell_type}__alt")) >= 1.0))

In [24]:
variants=variants.withColumn("any_emvar",
                    F.greatest( \
                        F.col("emVar_K562") , \
                        F.col("emVar_SKNSH"),  \
                        F.col("emVar_HepG2")
                    )
                   )

In [26]:
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#	display(variants.limit(20).toPandas())

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,K562__ref,HepG2__ref,SKNSH__ref,K562__alt,HepG2__alt,SKNSH__alt,K562__skew,HepG2__skew,SKNSH__skew,AC,AN,AF,cadd_phred,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,is_in_PLS,is_in_TF,is_in_CA-TF,P_ANNO,mean_ref,mean_skew,MAF,category,emVar_K562,emVar_SKNSH,emVar_HepG2,any_emvar
0,chr22,10526249,.,G,A,.,PASS,K562__ref=1.0380609;HepG2__ref=1.2221913;SKNSH...,1.038061,1.222191,1.692472,1.007452,1.186535,1.671206,-0.030609,-0.035656,-0.021266,1,124144,0.000008,6.019,False,True,False,False,False,False,False,False,0.076,1.317575,-0.029177,0.000008,SINGLETON,False,False,False,False
1,chr22,10562002,rs1248303998,C,A,.,PASS,K562__ref=0.4874192;HepG2__ref=0.13514678;SKNS...,0.487419,0.135147,0.390264,0.502203,0.064501,0.267999,0.014784,-0.070646,-0.122265,1,102180,0.000010,4.516,False,False,False,False,False,False,False,False,-0.148,0.337610,-0.059376,0.000010,SINGLETON,False,False,False,False
2,chr22,10562002,rs1248303998,C,T,.,PASS,K562__ref=0.4874192;HepG2__ref=0.13514678;SKNS...,0.487419,0.135147,0.390264,0.606181,0.038121,0.202303,0.118761,-0.097026,-0.187961,34366,90740,0.378730,5.194,False,False,False,False,False,False,False,False,-0.148,0.337610,-0.055409,0.378730,COMMON,False,False,False,False
3,chr22,10566028,.,G,T,.,PASS,K562__ref=0.15791912;HepG2__ref=0.13911162;SKN...,0.157919,0.139112,0.134537,0.118102,0.092486,0.097741,-0.039817,-0.046626,-0.036796,1,80190,0.000012,3.985,False,False,False,False,False,False,False,False,0.452,0.143856,-0.041080,0.000012,SINGLETON,False,False,False,False
4,chr22,10572715,rs1234361146,C,G,.,PASS,K562__ref=0.80345386;HepG2__ref=1.0439757;SKNS...,0.803454,1.043976,1.364886,0.692944,0.981769,1.301215,-0.110510,-0.062207,-0.063671,25,78044,0.000320,10.260,False,False,False,False,False,False,False,False,0.000,1.070772,-0.078796,0.000320,RARE,False,False,False,False
5,chr22,10572715,rs1234361146,C,T,.,PASS,K562__ref=0.80345386;HepG2__ref=1.0439757;SKNS...,0.803454,1.043976,1.364886,0.663573,0.953040,1.250090,-0.139880,-0.090936,-0.114795,1,78046,0.000013,10.710,False,False,False,False,False,False,False,False,0.000,1.070772,-0.115204,0.000013,SINGLETON,False,False,False,False
6,chr22,10573546,rs1365232471,G,C,.,PASS,K562__ref=0.5612022;HepG2__ref=0.13475315;SKNS...,0.561202,0.134753,-0.062847,0.666298,0.209002,-0.011229,0.105096,0.074249,0.051619,1,113428,0.000009,9.584,False,False,False,False,False,False,False,False,-0.195,0.211036,0.076988,0.000009,SINGLETON,False,False,False,False
7,chr22,10586962,.,A,T,.,PASS,K562__ref=-0.10138942;HepG2__ref=-0.25556168;S...,-0.101389,-0.255562,-0.338818,-0.104353,-0.260162,-0.362042,-0.002963,-0.004600,-0.023225,2,131730,0.000015,7.619,False,False,False,False,False,False,False,False,0.099,-0.231923,-0.010263,0.000015,ULTRARARE,False,False,False,False
8,chr22,10588238,rs1163776513,C,T,.,PASS,K562__ref=0.25078923;HepG2__ref=0.2657217;SKNS...,0.250789,0.265722,0.160320,0.263124,0.282678,0.201686,0.012335,0.016956,0.041366,2,124764,0.000016,4.002,False,False,False,False,False,False,False,False,0.126,0.225610,0.023552,0.000016,ULTRARARE,False,False,False,False
9,chr22,10588253,rs1223553271,A,G,.,PASS,K562__ref=0.3157834;HepG2__ref=0.30098882;SKNS...,0.315783,0.300989,0.213589,0.306289,0.299335,0.199092,-0.009495,-0.001654,-0.014498,6,127876,0.000047,5.724,False,False,False,False,False,False,False,False,0.155,0.276787,-0.008549,0.000047,ULTRARARE,False,False,False,False


In [ ]:
columns_to_keep=["CHROM","POS","ID","REF","ALT","any_emvar"]
variants = variants.select(columns_to_keep)

In [ ]:
output_root="/home/mcn26/varef/scripts/noon_data/emvar_alone/"

variants.repartition(1).write \
    .option("header","true") \
    .option("delimiter","\t") \
    .option("compression", "gzip") \
    .csv(output_root+chromosome)

spark.stop()